### Cargar modulos

In [1]:
import numpy as np
import pandas as pd

import sys    
reload(sys)  
sys.setdefaultencoding('utf8')

import seaborn as sns

## Cargar data

In [33]:

data = pd.read_csv('./../Country_data/Boschma2018FilteredDataset')

## SITC Sections
SITC4_full = pd.read_csv('./SITC4_full.csv')

### Ejemplito: Balanza argentina 2010 por grandes sectores

In [3]:
merged = data.merge(SITC4_full[['SITC ID 4', 'SITC label 1']].drop_duplicates(), left_on = 'pcode', right_on = 'SITC ID 4')

df = merged.loc[merged.ccode == 'arg'].loc[merged.year == 2010].groupby(['year', 'ccode', 'SITC label 1']).sum()[['export_val', 'import_val']]

df.style.bar(subset=['export_val', 'import_val'], color='#d65f5f')

## Exportaciones, importaciones y saldo

In [4]:
data['decada'] = data['year'].round(-1)
# data['import_val'] = - data['import_val']
data['saldo'] = data['export_val'] - data['import_val']
grouped = data.groupby(['decada', 'ccode', 'pcode'])[['export_val', 'import_val', 'saldo']].sum()/1000000./10.


In [5]:
def get_main_entries(df, column, pct = .8):
    sorted = df.sort_values(by = column, ascending = False)
    pct_cumsum = sorted[column].cumsum()/sorted[column].sum()
    sorted_sel = sorted[pct_cumsum < pct]

    return sorted_sel

pr = pd.read_csv('./products_sitc_rev2.tsv', delimiter = '\t')
pr.columns = ['id', 'SITC ID 4', 'SITC label 4']

## Explorador de balanzas comerciales
### Ver balanza eligiendo pais, decada o productos.  (millones de USD promedio anual)

In [22]:
import imgkit

decada = 2010
pais = 'arg'
# producto = 5417 # Medicamentos
ordenar_por = 'import_val'

df_selection = grouped.reset_index().loc[(df.decada == decada) & (df.ccode == pais)]

df_summary = get_main_entries(df_selection, ordenar_por, .9)
df_summary = df_selection.sort_values(by = 'saldo')

df_summary = df_summary.merge(pr[['SITC ID 4', 'SITC label 4']], left_on = 'pcode', right_on = 'SITC ID 4', how = 'left')
df_summary = df_summary[['decada', 'ccode', 'pcode','SITC label 4', 'export_val', 'import_val', 'saldo']]

In [47]:
# Muestra top 15

surplus = True

if surplus:
    label = 'sup'
    styled = df_summary.tail(10).style.bar(subset=['export_val', 'import_val', 'saldo'], align='mid', color=['#d65f5f', '#5fba7d'])
else:
    label = 'def'
    styled = df_summary.head(10).style.bar(subset=['export_val', 'import_val', 'saldo'], align='mid', color=['#d65f5f', '#5fba7d'])

imgkit.from_string(styled.render(), './'+str(pais)+'_'+str(decada)+'_'+str(label)+'.png')

True

In [46]:
styled.render()

u'<style  type="text/css" >\n    #T_0a45f3d2_8c5a_11e8_91f4_5ce0c5892623row0_col4 {\n            width:  10em;\n             height:  80%;\n            background:  linear-gradient(90deg, transparent 0%, transparent 0.0%, #5fba7d 0.0%, #5fba7d 12.7%, transparent 12.7%);\n            width:  10em;\n             height:  80%;\n            background:  linear-gradient(90deg, transparent 0%, transparent 0.0%, #5fba7d 0.0%, #5fba7d 12.7%, transparent 12.7%);\n        }    #T_0a45f3d2_8c5a_11e8_91f4_5ce0c5892623row0_col5 {\n            width:  10em;\n             height:  80%;\n            background:  linear-gradient(90deg, transparent 0%, transparent 0.0%, #5fba7d 0.0%, #5fba7d 0.4%, transparent 0.4%);\n            width:  10em;\n             height:  80%;\n            background:  linear-gradient(90deg, transparent 0%, transparent 0.0%, #5fba7d 0.0%, #5fba7d 0.4%, transparent 0.4%);\n        }    #T_0a45f3d2_8c5a_11e8_91f4_5ce0c5892623row0_col6 {\n            width:  10em;\n             h

In [43]:
html = styled_table.render()
import imgkit
imgkit.from_string(html, 'styled_table.png')

True

In [40]:
data.loc[data.year == 2010].groupby(['ccode']).sum()['export_val'].sort_values().tail(10)

ccode
gbr    3.476282e+11
can    3.620466e+11
nld    3.744516e+11
ita    3.880825e+11
kor    4.293415e+11
fra    4.867175e+11
jpn    6.820606e+11
deu    1.062556e+12
usa    1.102567e+12
chn    1.680028e+12
Name: export_val, dtype: float64